In [20]:
import os, glob, sys
import numpy as np
import json

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.utils import np_utils

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

MFCC_FEATURES_N = 12
CHROMA_N = 12

GENRES = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal',
          'pop', 'reggae', 'rock']
nb_classes = 10

In [21]:
def read_file(filename):
    with open(filename) as data_file:    
        data = json.load(data_file)
    return data

In [22]:
def json_to_df(filename):
    json_data = read_file(filename)
    song_dataset = list()
    
    for song_data in json_data:
        data_dict = dict()
        del song_data['song_name']
        
        for feature in song_data:
            if isinstance(song_data[feature],list):
                values = song_data[feature]
                for i in range(len(values)):
                    data_dict[((feature+"{}").format(i))] = values[i]
            else:
                data_dict[feature] = song_data[feature]
    
        data_dict['genre'] = GENRES.index(data_dict['genre'])
        
        song_dataset.append(data_dict.values())
        
    return np.array(song_dataset)

In [23]:
# read the data
df = json_to_df("../feature_data.json")
print df.shape

# shuffle the data
np.random.shuffle(df)

# split into input and corresponding labels
x_data = df[:, 1:]

y_data = df[:, 0]

# convert class vectors to binary class matrices
y_data = np_utils.to_categorical(y_data, nb_classes)

print x_data.shape
print y_data.shape

(10000, 30)
(10000, 29)
(10000, 10)


In [24]:
# divide into test and train data set
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=42)

scaler = preprocessing.StandardScaler().fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

print x_train.shape
print x_test.shape

print y_train.shape
print y_test.shape

(7000, 29)
(3000, 29)
(7000, 10)
(3000, 10)


In [25]:
def get_keras_model():
    model = Sequential()
    model.add(Dense(output_dim=35, input_dim=29))
    model.add(Activation('relu'))
    model.add(Dense(output_dim=10))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),metrics=['accuracy'])
    
    return model

In [ ]:
model = get_keras_model()

history = model.fit(x_train, y_train, nb_epoch=50, batch_size=32)
loss_and_metrics = model.evaluate(x_test, y_test, batch_size=32)

print loss_and_metrics

Epoch 1/50
7000/7000 [==============================] - 0s - loss: 1.8814 - acc: 0.3323      

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

clf = RandomForestClassifier(n_estimators=100, n_jobs=2)
clf = clf.fit(x_train, y_train)

y_pred = clf.predict(x_test)

score = accuracy_score(y_test, y_pred)

print score


0.329
